In [1]:
import sys
import midi
import numpy as np
import os


In [2]:
directories=['../dataset']
notes=midi.get_notes(directories)

number of samples:1399006
vocab size:72


In [3]:
network_in,network_out=midi.prepare_sequences(notes,is_LSTM=True)

In [7]:
pitchnames=sorted(set(item for item in notes))
print(pitchnames)

['', '48', '48,53', '48,60', '50', '50,62', '52', '52,60', '52,64', '53', '53,65', '54', '55', '55,60', '55,62', '55,64,72', '56', '57', '57,69', '58', '59', '59,71', '60', '60,72', '61', '62', '62,69', '62,74', '63', '64', '64,72', '64,76', '65', '65,69', '65,71', '65,77', '66', '67', '68', '68,71', '69', '69,72', '69,74', '69,81', '70', '71', '72', '72,74', '72,76', '73', '73,76', '74', '74,77', '74,79', '74,81', '75', '76', '76,79', '77', '77,81', '78', '79', '80', '80,83', '81', '82', '83', '84', '85', '86', '88', '89']


In [4]:
vocab_size=network_in.shape[-1]

In [6]:
from keras.models import Sequential
from keras.layers import Activation,Dense,LSTM,Dropout,Flatten,Bidirectional
model=Sequential()
model.add(LSTM(512,input_shape=(network_in.shape[1:]),return_sequences=True,name='LSTM layer 1'))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(512,return_sequences=True),name="LSTM layer"))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(512)))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(vocab_size))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.save('LSTMArchitecture.h5')

ValueError: 'LSTM layer 1_input' is not a valid scope name

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint
filepath='bestweight.hdf5'
checkpoint=ModelCheckpoint(filepath,monitor='loss',verbose=1,save_best_only=True)
history=model.fit(network_in,network_out,epochs=1,batch_size=100,validation_split=0.2,callbacks=[checkpoint])

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model train loss vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_acc'])
plt.title('model train accuracy vs validation accuracy')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
pitchnames=sorted(set(item for item in notes))
start=np.random.randint(0,len(network_in)-1)
int_to_note=dict((number,note) for number,note in enumerate(pitchnames))
pattern=network_input[start]
prediction_output=[]
for note_index in range(16*96):
    prediction_input=np.reshape(pattern,(1,len(pattern),1))
    prediction_input=prediction_input/float(n_vocab)
    prediction=model.predict(prediction_input,verbose=0)
    index=np.argmax(prediction)
    result=int_to_note[index]
    prediction_output.append(result)
    pattern=np.append(pattern,index)
    pattern=pattern[1:len(pattern)]
